In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row.filename)
        img = Image.open(img_path).convert('RGB')
        
        x, y = row.x * 3, row.y * 3

        x = x - 960 if x <= 960 else x - 960
        y = y - 540 if y <= 540 else y - 540
        
        if self.transform:
            img = self.transform(img)

        label = torch.tensor([x, y], dtype=torch.float32)
        
        return img, label

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((360, 640)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create dataset
csv_file = "classification_data/explore_img/explore_data.csv"
img_dir = "classification_data/explore_img/"
dataset = CustomDataset(csv_file, img_dir, transform=transform)

# Split dataset into training and validation sets (optional)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [3]:
class MousePositionCNN(nn.Module):
    def __init__(self):
        super(MousePositionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 45 * 80, 512)  # Adjust dimensions based on image size and pooling
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # Output layer with 2 values (x, y)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 45 * 80)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

model = MousePositionCNN().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [4]:
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation loop
model.eval()
with torch.no_grad():
    val_loss = 0.0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
    
    print(f'Validation Loss: {val_loss / len(val_loader):.4f}')

Epoch [1/8], Loss: 159719.5297
Epoch [2/8], Loss: 155720.8005
Epoch [3/8], Loss: 132400.7013
Epoch [4/8], Loss: 102687.4378
Epoch [5/8], Loss: 75837.7171
Epoch [6/8], Loss: 56001.8221
Epoch [7/8], Loss: 38152.0673
Epoch [8/8], Loss: 23599.9987
Validation Loss: 136795.5752


In [5]:
torch.save(model.state_dict(), 'ME_torch.pth')

In [6]:
ME_model = MousePositionCNN()
ME_model.load_state_dict(torch.load('ME_torch.pth'))

<All keys matched successfully>

In [7]:
num_epochs = 15
for epoch in range(num_epochs):
    ME_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation loop
ME_model.eval()
with torch.no_grad():
    val_loss = 0.0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
    
    print(f'Validation Loss: {val_loss / len(val_loader):.4f}')

Epoch [1/15], Loss: 18045.9690
Epoch [2/15], Loss: 10881.4392
Epoch [3/15], Loss: 7038.2654
Epoch [4/15], Loss: 5815.0833
Epoch [5/15], Loss: 5625.5950
Epoch [6/15], Loss: 4683.6297
Epoch [7/15], Loss: 4150.6272
Epoch [8/15], Loss: 3873.5159
Epoch [9/15], Loss: 3825.8692
Epoch [10/15], Loss: 3710.1301
Epoch [11/15], Loss: 3374.0087
Epoch [12/15], Loss: 3220.7778
Epoch [13/15], Loss: 3187.6920
Epoch [14/15], Loss: 3359.6978
Epoch [15/15], Loss: 3364.2711
Validation Loss: 127669.1191


In [8]:
torch.save(model.state_dict(), 'ME_torch.pth')